In [ ]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
from utils.MIND import MIND, MIND_news, MIND_history
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [ ]:
t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('google/bigbird-roberta-base', cache_dir=config.path + "bert_cache/")

In [ ]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'
# config.reducer = 'first'

# config.signal_length = 10
config.scale = 'large'
config.impr_size = 1000
config.mode = 'encode'
config.news = None
# config.fast = True
# config.news = 21563

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
# file_directory = manager.path + "MIND/MIND{}_{}/".format(manager.scale, manager.mode)
file_directory = manager.path + "MIND/MIND{}_dev/".format(manager.scale)
# manager.gather_same_user_impr()

a = MIND_history(manager, file_directory)
# b = MIND_news(manager)

In [ ]:
behaviors = pickle.load(open('data/cache/bert/MINDlarge_dev/1000/behaviors.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDdemo_train/news.pkl','rb'))
# news = pickle.load(open('data/cache/bert/MINDlarge_train/news.pkl','rb'))
# enc = news['encoded_news']

In [ ]:
# bm25, entity, first
index = 1

# news = t.convert_ids_to_tokens(a[index]['his_encoded_index'][1])
# mask = a[index]['his_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = a[index]['his_subword_index'][1]

# for i,j,k in zip(news,mask,subword):
#     print(i,j,k)
res = a[index]
ids = res['cdd_id'][1]
news = t.convert_ids_to_tokens(res['cdd_encoded_index'][1])
mask = res['cdd_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = res['cdd_subword_index'][1]
print(ids)
for i,j,k in zip(news,mask):
    print(i,j,k)

In [ ]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
# subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k in zip(news,mask,dedup):
    print(i,j,k)

In [17]:
import pickle
news = pickle.load(open('data/cache/bert/MINDlarge_dev/news.pkl','rb'))['encoded_news']
bm25 = pickle.load(open('data/cache/bert/MINDlarge_dev/bm25.pkl','rb'))['encoded_news']
entity = pickle.load(open('data/cache/bert/MINDlarge_dev/entity.pkl','rb'))['encoded_news']

bm25_indice = []
entity_indice = []

for n,b,e in zip(news, bm25, entity):
    # strip [CLS]
    bm25_index = []
    entity_index = []

    # for each token in the first 3 bm25, find its position in the original news
    for tok1 in b[1:4]:
        found = False
        for i,tok2 in enumerate(n[1:100]):
            if tok1 == tok2:
                bm25_index.append(i)
                found = True
                break
        if not found:
            bm25_index.append(-1)

    for tok1 in e[1:4]:
        found = False
        for i,tok2 in enumerate(n[1:100]):
            if tok1 == tok2:
                found = True
                entity_index.append(i)
                break
        if not found:
            entity_index.append(-1)

    bm25_indice.append(bm25_index)
    entity_indice.append(entity_index)

np.save("data/cache/kid/bm25.npy", np.asarray(bm25_indice))
np.save("data/cache/kid/entity.npy", np.asarray(entity_indice))

/tmp/ipykernel_20980/586002259.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save("data/cache/kid/bm25.npy", np.asarray(bm25_indice))
/tmp/ipykernel_20980/586002259.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save("data/cache/kid/entity.npy", np.asarray(entity_indice))


In [18]:
bm25_indice

[[0, 0, 0],
 [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  15,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  16,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  11],
 [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  8,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  18,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  19],
 [0, -1, -1, -1, -1, -1, -1, -1, 7, -1, -1, -1, -1, -1, -1, -1, -1, 8],
 [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  10,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  19,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  20],
 [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  

In [14]:
count = 0
for x in entity_indice:
    if len(x) == 0:
        count += 1
count